In [ ]:
!pip install --upgrade asyncpraw
!pip install pandas nltk spacy gensim pyLDAvis
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Importing necessary libraries

In [ ]:
import asyncpraw
import re
import asyncio
import pandas as pd
import nltk
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from collections import defaultdict
import os

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Setting up PRAW for data extraction from reddit based on the subqueries

In [ ]:
reddit = asyncpraw.Reddit(
    client_id="aJ70RPat4e9RcfO-rYDHng",
    client_secret="ToJhDX7RV6peJ70IKqtUQXy209x7Bg",
    user_agent="API for social media mining",  # Example: 'text_mining_script'
)

# Asynchronous function to check post relevancy
start_date = int(datetime.datetime(2019, 1, 1).timestamp())  # Start of 2019
end_date = int(datetime.datetime(2023, 12, 31).timestamp())
async def is_relevant(title):
    # Define more specific keywords or patterns related to startup success indicators
    patterns = [
        r"startup\s+success\s+factor",
        r"factors\s+for\s+startup\s+growth",
        r"predictors\s+of\s+startup\s+success",
        r"keys?\s+to\s+successful\s+ecommerce?",
        r"startup\s+(growth|scalability|traction)\s+factors",
        r"market\s+fit\s+for\s+startups?",
        r"funding\s+strategies\s+for\s+startups",
        r"customer\s+acquisition\s+for\s+startups",
        r"team\s+dynamics\s+in\s+startups?",
        r"entrepreneurial\s+(skills|traits)\s+for\s+startup\s+success"
    ]
    # Check if the title matches any of the refined regex patterns
    return any(re.search(pattern, title.lower()) for pattern in patterns)

# Asynchronous function to fetch posts and save them in a DataFrame
async def fetch_posts():
    subreddit = await reddit.subreddit('smallbusiness+Entrepreneur+startup+social media marketing')
    posts = subreddit.search("social media marketing tips for successful startup", limit=500)

    # List to store post data
    posts_data = []

    async for post in posts:
        if is_relevant(post.title) and start_date <= post.created_utc <= end_date:  # Filter posts based on relevancy
            # Append post data to the list
            posts_data.append({
                "Title": post.title,
                "Body": post.selftext,
                "Upvotes": post.score
            })

            df_new = pd.DataFrame(posts_data)

    # Check if the CSV file exists
    if os.path.isfile("filtered_posts1.csv"):
        # Load the existing data
        df_existing = pd.read_csv("filtered_posts1.csv")
        # Append new data to the existing DataFrame
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        # Drop duplicates in case of overlapping data
        df_combined.drop_duplicates(subset=['Title', 'Body'], inplace=True)
        # Save the combined DataFrame back to the CSV
        df_combined.to_csv("filtered_posts1.csv", index=False)
        print(f"Data appended to 'filtered_posts1.csv'. Total posts: {len(df_combined)}")
    else:
        # If the file doesn't exist, save the new data as a new CSV
        df_new.to_csv("filtered_posts1.csv", index=False)
        print(f"Data saved to 'filtered_posts.csv1'. Total posts: {len(df_new)}")

# Run the asynchronous function with a specific query
await fetch_posts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/lib/python3.10/asyncio/events.py:80: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)
<ipython-input-39-655ca2c57b91>:39: RuntimeWarning: coroutine 'is_relevant' was never awaited
  if is_relevant(post.title) and start_date <= post.created_utc <= end_date:  # Filter posts based on relevancy
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object 

Data appended to 'filtered_posts1.csv'. Total posts: 530


In [ ]:
df = pd.read_csv("filtered_posts.csv")

# Display the first few rows to verify the data
print(df.head())
print(df.shape)

                                               Title  \
0  7 Inspiring Micro-Startup Success Stories to F...   
1  7 Inspiring Micro-Startup Success Stories to F...   
2  I went from art-school dropout to 12k/month, b...   
3  What are the key factors a successful entrepre...   
4  15 lessons from our first $15 million in recur...   

                                                Body  Upvotes  
0  In this article, we embark on a fascinating jo...        5  
1  In this article, we embark on a fascinating jo...        1  
2  Hey there! so like it says in the title, I dro...      403  
3  Hi reddit. I am working on what I need to do t...        2  
4  Hey Reddit,\n\nI wrote this post for my blog, ...      900  
(567, 3)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(reddit.auth.limits)

{'remaining': 982.0, 'reset_timestamp': 1731367200.197238, 'used': 18}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Data preprocessing and applying LDA for topic modeling

In [ ]:
custom_stopwords = {'look', 'good', 'want', 'need', 'know', 'like', 'really', 'think', 'thing', 'stuff'}
stop_words.update(custom_stopwords)
nlp = spacy.load("en_core_web_sm")

# Combine title and body into a single text column for context
df['Text'] = df['Title'] + ' ' + df['Body']

# Step 3: Define a Preprocessing Function with Enhanced Filtering
def preprocess_text(text):
    # Remove URLs, special characters, and digits
    text = re.sub(r'http\S+|www\S+|https\S+|@\w+|[^A-Za-z\s]', '', text, flags=re.MULTILINE)
    # Convert to lowercase
    text = text.lower()
    # Tokenize and Lemmatize using SpaCy, keeping only nouns, verbs, and adjectives
    doc = nlp(text)
    tokens = []
    for token in doc:
        # Filter by part of speech (POS) and stopwords
        if token.text not in stop_words and not token.is_stop and not token.is_punct and not token.like_num:
            if token.pos_ in ['NOUN', 'VERB', 'ADJ']:  # Keep only nouns, verbs, and adjectives
                tokens.append(token.lemma_)
    return ' '.join(tokens)

# Step 4: Apply the Preprocessing Function
df['Processed_Text'] = df['Text'].apply(preprocess_text)

# Step 5: TF-IDF Transformation
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=2, stop_words='english', ngram_range=(1, 2))

# Fit and transform the processed text data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Processed_Text'])

# Get feature names to filter high-frequency terms based on TF-IDF scores
feature_names = tfidf_vectorizer.get_feature_names_out()
average_tfidf_scores = tfidf_matrix.mean(axis=0).A1  # Average TF-IDF score per feature (term)
tfidf_scores = dict(zip(feature_names, average_tfidf_scores))

# Set a threshold for high-frequency terms based on TF-IDF score
tfidf_threshold = 0.1  # Adjust this threshold based on your data
low_tfidf_features = [word for word, score in tfidf_scores.items() if score < tfidf_threshold]

# Step 6: Filter Processed Texts Based on TF-IDF
filtered_texts = []
for text in df['Processed_Text']:
    filtered_text = ' '.join([word for word in text.split() if word in low_tfidf_features])
    filtered_texts.append(filtered_text)

# Tokenize the filtered text data for Gensim
texts = [text.split() for text in filtered_texts]

# Step 7: Create Dictionary and Corpus for Gensim
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

# Step 8: Train the LDA Model using Gensim
num_topics = 5  # Number of topics to generate
lda_model = gensim.models.LdaModel(corpus=corpus,
                                   id2word=id2word,
                                   num_topics=num_topics,
                                   random_state=42,
                                   passes=10)

# Display the Topics and Top Words in Each Topic
print("LDA Topics:")
for i, topic in lda_model.print_topics(num_topics=num_topics, num_words=10):
    print(f"Topic {i + 1}: {topic}")

# Create the pyLDAvis panel for Gensim LDA model
panel = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='tsne')

# Display the visualization
pyLDAvis.display(panel)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LDA Topics:
Topic 1: 0.012*"work" + 0.009*"people" + 0.009*"time" + 0.008*"year" + 0.007*"start" + 0.007*"project" + 0.006*"job" + 0.006*"business" + 0.006*"build" + 0.005*"book"
Topic 2: 0.017*"business" + 0.012*"product" + 0.010*"work" + 0.009*"time" + 0.008*"company" + 0.008*"people" + 0.007*"idea" + 0.006*"customer" + 0.006*"start" + 0.006*"build"
Topic 3: 0.016*"business" + 0.011*"link" + 0.009*"work" + 0.008*"time" + 0.007*"content" + 0.007*"website" + 0.006*"customer" + 0.006*"loan" + 0.005*"thing" + 0.005*"video"
Topic 4: 0.016*"content" + 0.012*"post" + 0.011*"marketing" + 0.010*"ad" + 0.008*"create" + 0.008*"people" + 0.008*"product" + 0.007*"page" + 0.007*"seo" + 0.007*"social"
Topic 5: 0.016*"business" + 0.013*"start" + 0.013*"work" + 0.012*"product" + 0.010*"people" + 0.010*"time" + 0.009*"company" + 0.008*"customer" + 0.008*"build" + 0.007*"year"


Getting the top 20 words from the corpus based on weight of the word in corpus.

In [ ]:
num_words_per_topic = 10  # Number of top words to extract per topic

# Step 1: Extract Top Words per Topic
topic_words = defaultdict(float)  # Dictionary to store each word's highest weight across topics

# Loop through each topic in the LDA model
for topic_idx, topic in lda_model.show_topics(formatted=False, num_words=num_words_per_topic):
    print(f"Topic {topic_idx + 1}:")
    for word, weight in topic:
        topic_words[word] = max(topic_words[word], weight)  # Keep the maximum weight for each word

# Step 2: Sort Words by Weight
sorted_words = sorted(topic_words.items(), key=lambda x: x[1], reverse=True)

# Step 3: Display Top Highest-Rated Words
top_n = 20  # Number of top highest-rated words you want to display
top_highest_rated_words = sorted_words[:top_n]

print("\nTop Highest-Rated Words Across All Topics:")
for word, weight in top_highest_rated_words:
    print(f"{word}: {weight:.4f}")

Topic 1:
Topic 2:
Topic 3:
Topic 4:
Topic 5:

Top Highest-Rated Words Across All Topics:
business: 0.0167
content: 0.0161
start: 0.0129
work: 0.0128
product: 0.0123
post: 0.0116
link: 0.0113
marketing: 0.0110
people: 0.0103
ad: 0.0097
time: 0.0097
company: 0.0091
create: 0.0083
year: 0.0078
customer: 0.0078
build: 0.0075
page: 0.0073
seo: 0.0072
social: 0.0070
project: 0.0070


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Getting the contexts for the highest rated words for understanding the factors for startup success.

In [ ]:
contexts_data = []
# Define the top words (replace this list with your top highest-rated words)
top_words = ["business", "content", "start", "work", "product", "post", "link",
    "marketing", "people", "ad", "time", "company", "create", "year",
    "customer", "build", "page", "seo", "social", "project"]

word_contexts = defaultdict(list)

# Extract unique contexts for each top word in the 'Processed_Text' column
for index, row in df.iterrows():
    processed_text = row['Processed_Text']  # Use the processed, cleaned text column
    for word in top_words:
        # Find all unique occurrences of the word in the processed text
        unique_contexts = set()  # Use a set to store unique contexts for this word in this document
        for match in re.finditer(r'\b' + re.escape(word) + r'\b', processed_text, re.IGNORECASE):
            # Extract a window of words around the matched word for context
            start = max(0, match.start() - 50)  # 50 characters before the word
            end = min(len(processed_text), match.end() + 50)  # 50 characters after the word
            context = processed_text[start:end]
            unique_contexts.add(context)  # Add the context to the set to avoid duplicates within this document

        for context in unique_contexts:
            contexts_data.append({
                "Document_Index": index,
                "Word": word,
                "Context": context
            })

# Convert the list of contexts to a DataFrame
contexts_df = pd.DataFrame(contexts_data)

# Limit to first 5 contexts per word
limited_contexts_df = contexts_df.groupby("Word").head(5)

print(limited_contexts_df)
limited_contexts_df.to_csv("limited_top_word_contexts.csv", index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


      Document_Index      Word  \
0                  0  business   
1                  0  business   
2                  0  business   
3                  0   content   
4                  0   content   
...              ...       ...   
1277              21      link   
1456              22      link   
1514              22       seo   
1515              22       seo   
1516              22       seo   

                                                Context  
0      customer experience textmagic marketing softw...  
1     user innovative approach simplify billing proc...  
2     instream clear pricing structure start year ca...  
3     edium platform user save time increase efficie...  
4     rship source code earn proof repurposeio autom...  
...                                                 ...  
1277   mobile phone initial vision want audio clicka...  
1456  advantage allow refine process editorial calen...  
1514  y seo service personal injury lawyer stat prod...  
1515  e use

In [ ]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Title,Body,Upvotes,Text,Processed_Text
0,7 Inspiring Micro-Startup Success Stories to F...,"In this article, we embark on a fascinating jo...",5,7 Inspiring Micro-Startup Success Stories to F...,inspire microstartup success story fuel entrep...
1,7 Inspiring Micro-Startup Success Stories to F...,"In this article, we embark on a fascinating jo...",1,7 Inspiring Micro-Startup Success Stories to F...,inspire microstartup success story fuel entrep...
2,"I went from art-school dropout to 12k/month, b...","Hey there! so like it says in the title, I dro...",403,"I went from art-school dropout to 12k/month, b...",go artschool dropout kmonth entrepreneur say t...
3,What are the key factors a successful entrepre...,Hi reddit. I am working on what I need to do t...,2,What are the key factors a successful entrepre...,key factor successful entrepreneur possess hi ...
4,15 lessons from our first $15 million in recur...,"Hey Reddit,\n\nI wrote this post for my blog, ...",900,15 lessons from our first $15 million in recur...,lesson recur revenue reddit write post blog th...
...,...,...,...,...,...
562,Traction is no longer king. Having the right t...,"This applies for any founder, but is particula...",196,Traction is no longer king. Having the right t...,traction king right team place valuable apply ...
563,How This Letter Made $2 Billion Dollars for Th...,"It's 1975.\n\nA copywriter, Martin Conroy, sit...",1061,How This Letter Made $2 Billion Dollars for Th...,letter dollar copywriter sit kitchen rack brai...
564,YC startup: Our app was so bad it went viral,I’m one of the cofounders of [Superpowered](ht...,351,YC startup: Our app was so bad it went viral I...,startup app bad go viral cofounder superpowere...
565,"Some ""lessons"" from someone who's sold $10m on...","Hi there, I am about 7 years into my entrepren...",494,"Some ""lessons"" from someone who's sold $10m on...",lesson sell year entrepreneur journey lurker s...


In [ ]:
df.duplicated().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0